In [1]:
!pip install pcompress

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/home/linuxbrew/.linuxbrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [8]:
!cargo install pcompress

    Updating crates.io index
  Downloaded pcompress v0.2.2                                                 
  Downloaded 1 crate (5.8 KB) in 0.42s                                     
  Installing pcompress v0.2.2
   Compiling proc-macro2 v1.0.28
   Compiling version_check v0.9.3
   Compiling unicode-xid v0.2.2
   Compiling syn v1.0.75
   Compiling libc v0.2.100
   Compiling cc v1.0.69
   Compiling serde v1.0.129
   Compiling unicode-segmentation v1.8.0
   Compiling unicode-width v0.1.8
   Compiling ryu v1.0.5
   Compiling ansi_term v0.11.0
   Compiling serde_json v1.0.66
   Compiling strsim v0.8.0
   Compiling bitflags v1.3.2
   Compiling vec_map v0.8.2
   Compiling lazy_static v1.4.0
   Compiling itoa v0.4.8
   Compiling textwrap v0.11.0          ] 0/47: version_check, unicode-wi...
   Compiling proc-macro-error-attr v1.0.46/47: version_check, unicode-se...
   Compiling proc-macro-error v1.0.4
   Compiling heck v0.3.3              ] 20/47: proc-macro2, unicode-segm...
   Compiling li

In [1]:
import pcompress

## GerryChain Quickstart

In [2]:
from gerrychain import (
    GeographicPartition,
    Graph,
    MarkovChain,
    proposals,
    updaters,
    constraints,
    accept,
)
from gerrychain.proposals import recom
from functools import partial
import tqdm

graph = Graph.from_json("../../examples/PA_VTDs.json")

my_updaters = {"population": updaters.Tally("TOTPOP", alias="population")}
initial_partition = GeographicPartition(
    graph, assignment="CD_2011", updaters=my_updaters
)

ideal_population = sum(initial_partition["population"].values()) / len(
    initial_partition
)

# We use functools.partial to bind the extra parameters (pop_col, pop_target, epsilon, node_repeats)
# of the recom proposal.
proposal = partial(
    recom, pop_col="TOTPOP", pop_target=ideal_population, epsilon=0.02, node_repeats=2
)

compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]), 2 * len(initial_partition["cut_edges"])
)

pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, 0.02)

chain = MarkovChain(
    proposal=proposal,
    constraints=[pop_constraint],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=10,
)

## Demo Usage

In [3]:
for partition in chain:  # normal
    print(partition.population)

{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 705782, 15: 705549, 7: 706391, 16: 705629, 11: 705883, 4: 705669, 8: 705689, 17: 705830, 18: 705847, 14: 705526, 13: 705028, 2: 705689, 1: 705588}
{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 692987, 15: 705549, 7: 719186, 16: 705629, 11: 705883, 4: 705669, 8: 705689, 17: 705830, 18: 705847, 14: 705526, 13: 705028, 2: 705689, 1: 705588}
{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 692987, 15: 705549, 7: 719186, 16: 705629, 11: 717249, 4: 705669, 8: 705689, 17: 694464, 18: 705847, 14: 705526, 13: 705028, 2: 705689, 1: 705588}
{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 692987, 15: 705549, 7: 719186, 16: 705629, 11: 717249, 4: 705669, 8: 705689, 17: 694464, 18: 705847, 14: 705526, 13: 705028, 2: 705689, 1: 705588}
{3: 702241, 5: 706258, 10: 705327, 9: 705679, 12: 708774, 6: 692987, 15: 705549, 7: 719186, 16: 705629, 11: 717249, 4: 705669, 8: 705689, 17: 694464, 18: 705847, 14: 70

In [5]:
partition.assignment.to_series()

411     3
2455    3
2456    3
2458    3
2459    3
       ..
6138    1
6139    1
6140    1
6141    1
6142    1
Length: 9255, dtype: int64

In [10]:
partition.flip({411:2}).assignment.to_series().at[411]

2

In [9]:
for partition in pcompress.Record(chain, "run.chain", executable="pv", extreme=True):
    print(partition.population)

{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 705782, 15: 705549, 7: 706391, 16: 705629, 11: 705883, 4: 705669, 8: 705689, 17: 705830, 18: 705847, 14: 705526, 13: 705028, 2: 705689, 1: 705588}
{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 705782, 15: 705549, 7: 706391, 16: 705629, 11: 694590, 4: 705669, 8: 705689, 17: 717123, 18: 705847, 14: 705526, 13: 705028, 2: 705689, 1: 705588}
{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 705782, 15: 705549, 7: 702017, 16: 705629, 11: 694590, 4: 705669, 8: 705689, 17: 717123, 18: 705847, 14: 705526, 13: 709402, 2: 705689, 1: 705588}
{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 705782, 15: 705549, 7: 702017, 16: 705629, 11: 694590, 4: 705669, 8: 705689, 17: 717123, 18: 705847, 14: 705526, 13: 718226, 2: 705689, 1: 696764}
{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 705782, 15: 705549, 7: 702017, 16: 705629, 11: 694590, 4: 705669, 8: 705689, 17: 717123, 18: 705847, 14: 70

## Replay

In [10]:
for partition in pcompress.Replay(graph, "run.chain", executable="pv", updaters=my_updaters):
    print(partition.population)

{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 705782, 15: 705549, 7: 706391, 16: 705629, 11: 705883, 4: 705669, 8: 705689, 17: 705830, 18: 705847, 14: 705526, 13: 705028, 2: 705689, 1: 705588}
{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 705782, 15: 705549, 7: 706391, 16: 705629, 11: 694590, 4: 705669, 8: 705689, 17: 717123, 18: 705847, 14: 705526, 13: 705028, 2: 705689, 1: 705588}
{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 705782, 15: 705549, 13: 709402, 16: 705629, 11: 694590, 4: 705669, 8: 705689, 17: 717123, 18: 705847, 14: 705526, 7: 702017, 2: 705689, 1: 705588}
{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 705782, 15: 705549, 1: 696764, 16: 705629, 11: 694590, 4: 705669, 8: 705689, 17: 717123, 18: 705847, 14: 705526, 7: 702017, 2: 705689, 13: 718226}
{3: 705317, 5: 706258, 10: 705327, 9: 705679, 12: 705698, 6: 705782, 15: 705549, 13: 705162, 16: 705629, 11: 694590, 4: 705669, 8: 705689, 17: 717123, 18: 705847, 14: 7